In [135]:
import pandas as pd
import numpy as np
import time
import math
import talib
from sklearn import preprocessing

# 讀取資料

In [211]:
fund_data = pd.read_csv("fundNav.csv", encoding="big5")

In [212]:
fund_data.columns = ['code', 'date', 'price', 'currency']

In [213]:
unique_code = np.unique(fund_data["code"])

In [214]:
len(unique_code)

500

In [215]:
unique_code[0]

'Fund_001'

# 分群

In [216]:
group_fund_data = fund_data.groupby("code")

In [217]:
test = group_fund_data.get_group(unique_code[0])

In [218]:
test.head()

,code,date,price,currency
0,Fund_001,20130701,17.50,美元
1,Fund_001,20130702,17.62,美元
2,Fund_001,20130703,17.58,美元
3,Fund_001,20130704,17.60,美元
4,Fund_001,20130705,17.60,美元


# 將500檔基金的日期合併(以日期最多的基金為基準)

### 先找出日期最多的基金

In [219]:
max_len = len(group_fund_data.get_group(unique_code[0]))
max_len_index = 0
for i in range(1,500):
    loop_len = len(group_fund_data.get_group(unique_code[i]))
    if(loop_len > max_len):
        max_len = loop_len
        max_len_index = i

In [220]:
max_len

1454

In [221]:
max_len_index

92

### 將第93檔基金放到unique_code的第一個，依序將其他放入

In [222]:
revise_unique_code = []
revise_unique_code.append(unique_code[92])

In [223]:
for i in range(0,92):
    revise_unique_code.append(unique_code[i])
for i in range(93,500):
    revise_unique_code.append(unique_code[i])

In [224]:
revise_unique_code[0:5]

['Fund_093', 'Fund_001', 'Fund_002', 'Fund_003', 'Fund_004']

### 以第93檔基金的日期做為基準，並合併

In [225]:
outside_data = group_fund_data.get_group(revise_unique_code[0])
outside_data.index = outside_data.date
outside_data = outside_data.iloc[:,2:3]
outside_data.columns = [revise_unique_code[0]]

In [226]:
outside_data.head()

,Fund_093
date,
20130701,1043.0
20130702,1057.0
20130703,1058.0
20130704,1051.0
20130705,1072.0


In [227]:
for i in range(1, len(revise_unique_code)):
    loop_data = group_fund_data.get_group(revise_unique_code[i])
    loop_data.index = loop_data.date
    loop_data = loop_data.iloc[:,2:3]
    loop_data.columns = [revise_unique_code[i]]
    outside_data = pd.concat([outside_data, loop_data], axis=1, join_axes=[outside_data.index])

In [228]:
merge_fund_data = outside_data.dropna()

In [229]:
merge_fund_data["date"] = merge_fund_data.index

C:\Users\a2224\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [230]:
merge_fund_data.index = range(0, len(merge_fund_data))

In [231]:
merge_fund_data.head()

,Fund_093,Fund_001,Fund_002,Fund_003,Fund_004,Fund_005,Fund_006,Fund_007,Fund_008,Fund_009,...,Fund_492,Fund_493,Fund_494,Fund_495,Fund_496,Fund_497,Fund_498,Fund_499,Fund_500,date
0,1057.0,17.82,9.35,10.48,204.1597,14.69,92.28,102.93,68.06,2.5376,...,23.23,11.50,114.91,15.25,6.94,18.64,16.47,10.16,21.59,20130708
1,1083.0,18.07,9.32,10.51,204.1607,14.69,93.00,104.04,69.30,2.5701,...,23.28,11.54,114.95,15.32,6.98,18.82,16.70,10.22,21.62,20130710
2,1080.0,18.07,9.36,10.53,204.1613,14.79,94.38,106.58,73.82,2.5690,...,23.35,11.57,115.17,15.13,7.11,19.11,17.08,10.30,21.70,20130711
3,1096.0,18.36,9.35,10.54,204.1618,14.79,94.45,107.30,72.79,2.5876,...,23.47,11.58,115.26,15.17,7.10,19.15,17.05,10.34,21.72,20130712
4,1106.0,18.47,9.38,10.57,204.1638,14.88,94.52,108.20,75.36,2.5952,...,23.60,11.61,115.57,15.21,7.15,19.20,17.05,10.40,21.84,20130716


# 讀取總經資料

### 原資料

In [232]:
basic_data = merge_fund_data.iloc[:,0:1]

In [233]:
basic_data.head()

,Fund_093
0,1057.0
1,1083.0
2,1080.0
3,1096.0
4,1106.0


In [234]:
date_basic = []
for i in range(0, len(merge_fund_data["date"])):
    date_basic.append(str(int(merge_fund_data["date"][i])))

In [235]:
basic_data.index = date_basic

In [236]:
basic_data.head()

,Fund_093
20130708,1057.0
20130710,1083.0
20130711,1080.0
20130712,1096.0
20130716,1106.0


### 石油

In [237]:
oil_data = pd.read_csv("DCOILBRENTEU-1.csv")

In [238]:
date_oil = []
for i in range(0, len(oil_data["DATE"])):
    date_oil.append(oil_data["DATE"][i].replace('-', ''))

In [239]:
oil_data.index = date_oil

In [240]:
oil_data = oil_data.drop(["DATE"], axis=1)

In [241]:
oil_data.head()

,DCOILBRENTEU
20130701,103.19
20130702,103.96
20130703,106.12
20130704,.
20130705,107.46


### 美國公債十年利率

In [242]:
dgs10_data = pd.read_csv("DGS10.csv")

In [243]:
date_dgs10 = []
for i in range(0, len(dgs10_data["observation_date"])):
    date_dgs10.append(dgs10_data["observation_date"][i].replace('-', ''))

In [244]:
dgs10_data.index = date_dgs10

In [245]:
dgs10_data = dgs10_data.drop(["observation_date"], axis=1)

In [246]:
dgs10_data.head()

,DGS10
20130701,2.50
20130702,2.48
20130703,2.52
20130704,NaN
20130705,2.73


### 合併

In [247]:
basic_data = pd.concat([basic_data, oil_data], axis=1, join_axes=[basic_data.index])

In [248]:
basic_data = pd.concat([basic_data, dgs10_data], axis=1, join_axes=[basic_data.index])

In [249]:
basic_data = basic_data.drop(["Fund_093"], axis=1)

In [251]:
basic_data.head()

,DCOILBRENTEU,DGS10
20130708,107.75,2.65
20130710,108.43,2.70
20130711,108.18,2.60
20130712,109.03,2.61
20130716,109.29,2.55


In [252]:
merge_fund_data.index = date_basic

# 訓練集資料

### 正規化

In [309]:
def normalize(df):
    newdf= df.copy()
    min_max_scaler = preprocessing.MinMaxScaler()
    
    newdf['close'] = min_max_scaler.fit_transform(df.close.values.reshape(-1,1))
    newdf['RSI'] = min_max_scaler.fit_transform(df.RSI.values.reshape(-1,1))
    newdf['MA'] = min_max_scaler.fit_transform(df.MA.values.reshape(-1,1))
    newdf['EMA'] = min_max_scaler.fit_transform(df.EMA.values.reshape(-1,1))
    newdf['WMA'] = min_max_scaler.fit_transform(df.WMA.values.reshape(-1,1))
    newdf['CMO'] = min_max_scaler.fit_transform(df.CMO.values.reshape(-1,1))
    newdf['MACD'] = min_max_scaler.fit_transform(df.MACD.values.reshape(-1,1))
    newdf['ROC'] = min_max_scaler.fit_transform(df.ROC.values.reshape(-1,1))
    newdf['DCOILBRENTEU'] = min_max_scaler.fit_transform(df.DCOILBRENTEU.values.reshape(-1,1))
    newdf['DGS10'] = min_max_scaler.fit_transform(df.DGS10.values.reshape(-1,1))
    return(newdf)

### 計算技術指標以及加入總經因子

In [293]:
merge_fund_data.iloc[:,0:1].head()

,Fund_093
20130708,1057.0
20130710,1083.0
20130711,1080.0
20130712,1096.0
20130716,1106.0


In [296]:
merge_fund_data.iloc[:,499:500].head()

,Fund_500
20130708,21.59
20130710,21.62
20130711,21.70
20130712,21.72
20130716,21.84


In [313]:
def pick_data_train(merge_data, basic_data, start_num, end_num):
    fun_data = merge_fund_data.iloc[:,start_num:end_num]
    fun_data.columns = ["close"]
    fun_data["RSI"] = talib.RSI(np.array(fun_data["close"]))
    fun_data["MA"] = talib.SMA(np.array(fun_data["close"]))
    fun_data["EMA"] = talib.EMA(np.array(fun_data["close"]))
    fun_data["WMA"] = talib.WMA(np.array(fun_data["close"]))
    fun_data["CMO"] = talib.CMO(np.array(fun_data["close"]))
    macd, macdsignal, macdhist = talib.MACD(np.array(fun_data["close"]), fastperiod=12, slowperiod=26, signalperiod=20)
    fun_data["MACD"] = macdhist
    fun_data["ROC"] = talib.ROC(np.array(fun_data["close"]))
    fun_data = pd.concat([fun_data, basic_data], axis=1, join_axes=[fun_data.index])
    fun_data = fun_data.dropna()
    fun_data.index = range(0, len(fun_data))
    fun_data = normalize(fun_data)
    fun_data = fun_data[0:726]
    return(fun_data)

In [314]:
def pick_data_test(merge_data, basic_data, start_num, end_num):
    fun_data = merge_fund_data.iloc[:,start_num:end_num]
    fun_data.columns = ["close"]
    fun_data["RSI"] = talib.RSI(np.array(fun_data["close"]))
    fun_data["MA"] = talib.SMA(np.array(fun_data["close"]))
    fun_data["EMA"] = talib.EMA(np.array(fun_data["close"]))
    fun_data["WMA"] = talib.WMA(np.array(fun_data["close"]))
    fun_data["CMO"] = talib.CMO(np.array(fun_data["close"]))
    macd, macdsignal, macdhist = talib.MACD(np.array(fun_data["close"]), fastperiod=12, slowperiod=26, signalperiod=20)
    fun_data["MACD"] = macdhist
    fun_data["ROC"] = talib.ROC(np.array(fun_data["close"]))
    fun_data = pd.concat([fun_data, basic_data], axis=1, join_axes=[fun_data.index])
    fun_data = fun_data.dropna()
    fun_data.index = range(0, len(fun_data))
    fun_data = normalize(fun_data)
    fun_data = fun_data[726:len(fun_data)]
    return(fun_data)

In [311]:
pick_data_train(merge_fund_data, basic_data, 0, 1).head()

,close,RSI,MA,EMA,WMA,CMO,MACD,ROC,DCOILBRENTEU,DGS10
0,1093.0,46.990162,1097.933333,1107.109595,1106.827957,-6.019675,7.674343,-2.236136,109.42,2.66
1,1079.0,43.397909,1097.933333,1105.296073,1105.606452,-13.204181,3.481116,-3.746655,109.66,2.65
2,1079.0,43.397909,1096.966667,1103.599552,1104.384946,-13.204181,0.374171,-3.746655,110.56,2.66
3,1097.0,49.189852,1096.333333,1103.173774,1104.387097,-1.620295,-0.567911,-2.140946,109.02,2.68
4,1111.0,53.200999,1097.166667,1103.678692,1105.333333,6.401999,-0.234351,-1.068566,111.63,2.71


In [312]:
normalize(pick_data_train(merge_fund_data, basic_data, 0, 1)).head()

C:\Users\a2224\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,close,RSI,MA,EMA,WMA,CMO,MACD,ROC,DCOILBRENTEU,DGS10
0,0.016471,0.420790,0.002084,0.005209,0.003144,0.420790,0.693180,0.412916,0.942060,0.786585
1,0.000000,0.366982,0.002084,0.002809,0.001572,0.366982,0.630257,0.374744,0.944771,0.780488
2,0.000000,0.366982,0.000825,0.000563,0.000000,0.366982,0.583635,0.374744,0.954936,0.786585
3,0.021176,0.453739,0.000000,0.000000,0.000003,0.453739,0.569499,0.415322,0.937542,0.798780
4,0.037647,0.513821,0.001086,0.000668,0.001220,0.513821,0.574504,0.442422,0.967021,0.817073


### 建立訓練集資料

In [307]:
def buildTrain(merge_data, basic_data, pastDay=30, futureDay=1):
    X_train, Y_train = [], []
    for j in range(0, 499):
        fun_data = pick_data_train(merge_data, basic_data, j, j+1)
        for i in range(len(fun_data)-futureDay-pastDay):
            X_train.append(np.array(fun_data.iloc[i:i+pastDay]))
            Y_train.append(np.array(fun_data.iloc[i+pastDay:i+pastDay+futureDay]["close"]))
        print(j)
    return np.array(X_train), np.array(Y_train)

In [259]:
test_train = normalize(pick_data(merge_fund_data, basic_data, 0, 1))

C:\Users\a2224\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [260]:
len(test_train)

764

In [261]:
buildTrain(test_train)[0]

array([[[1.64705882e-02, 4.20789951e-01, 2.08441897e-03, ...,
         4.12916158e-01, 9.42060086e-01, 7.41379310e-01],
        [0.00000000e+00, 3.66982217e-01, 2.08441897e-03, ...,
         3.74743749e-01, 9.44770725e-01, 7.35632184e-01],
        [0.00000000e+00, 3.66982217e-01, 8.25082508e-04, ...,
         3.74743749e-01, 9.54935622e-01, 7.41379310e-01],
        ...,
        [1.16470588e-01, 7.08485825e-01, 4.49452840e-02, ...,
         6.17010477e-01, 9.83171448e-01, 8.16091954e-01],
        [9.88235294e-02, 5.96261851e-01, 4.58572173e-02, ...,
         5.94868659e-01, 9.85543257e-01, 8.44827586e-01],
        [9.17647059e-02, 5.55383629e-01, 4.73771061e-02, ...,
         5.18409229e-01, 9.65552293e-01, 8.67816092e-01]],

       [[0.00000000e+00, 3.66982217e-01, 2.08441897e-03, ...,
         3.74743749e-01, 9.44770725e-01, 7.35632184e-01],
        [0.00000000e+00, 3.66982217e-01, 8.25082508e-04, ...,
         3.74743749e-01, 9.54935622e-01, 7.41379310e-01],
        [2.11764706e-02, 

In [281]:
xx = pick_data(merge_fund_data, basic_data, 0, 1)

In [282]:
xx["date"] = xx.index

In [283]:
xx.index = range(0, len(xx))

In [303]:
xx[726:len(xx)]

,close,RSI,MA,EMA,WMA,CMO,MACD,ROC,DCOILBRENTEU,DGS10,date
726,1822.0,63.786839,1755.533333,1780.814999,1771.212903,27.573679,19.453918,3.229462,73.14,2.97,20180502
727,1829.0,65.419527,1759.100000,1783.923709,1775.952688,30.839054,18.981278,4.216524,77.37,2.97,20180511
728,1839.0,67.662528,1762.566667,1787.477018,1781.107527,35.325056,18.808130,4.074703,78.17,3.00,20180514
729,1845.0,68.963325,1766.533333,1791.188178,1786.425806,37.926651,18.543523,3.826674,78.94,3.08,20180515
730,1853.0,70.658112,1770.766667,1795.176038,1792.004301,41.316224,18.346492,3.577418,78.19,3.09,20180516
731,1862.0,72.478835,1774.300000,1799.487261,1797.890323,44.957670,18.255721,3.848299,80.09,3.11,20180517
732,1869.0,73.838580,1778.033333,1803.971954,1804.000000,47.677159,18.078917,3.545706,78.38,3.06,20180518
733,1838.0,59.757622,1780.866667,1806.167312,1807.868817,19.515245,15.084336,0.657174,78.69,3.01,20180523
734,1807.0,49.576233,1782.933333,1806.221033,1809.554839,-0.847534,10.168605,-1.040526,78.9,2.98,20180524
735,1745.0,36.267291,1783.066667,1802.271289,1807.107527,-27.465419,1.794650,-4.331140,75.89,2.84,20180530
